In [365]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

#setting environment variables 
os.environ['SPOTIPY_CLIENT_ID'] = "replace me"
os.environ['SPOTIPY_CLIENT_SECRET'] = "replace me"
os.environ['SPOTIPY_REDIRECT_URI'] = "http://localhost:8888/"


## getting API params from knobs and sliders
danceability=1 #most danceable
valance=1 #1 is most happy
instrumentalness= 1 #most probable that there's no vocal
popularity = 80


## Authentication
cid ="replace me" 
secret = "replace me"
username = "replace me"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private user-read-playback-state user-modify-playback-state user-top-read'

token = util.prompt_for_user_token(username, scope)
#print("token is")
#print(token)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)
    

In [366]:
def get_current_user_top_artists_uri():
    tops = sp.current_user_top_tracks(limit=2)
    #print(tops['items'][0])
    return (tops['items'][0]['artists'][0]['uri'])
    
def get_current_user_top_track_genre():
    tops = sp.current_user_top_tracks(limit=2)
    artist_id = (tops['items'][0]['artists'][0]['id'])
    return(sp.artist(artist_id)['genres'])

def get_current_user_top_track_uri():
    tops = sp.current_user_top_tracks(limit=2)
    top_track_uri = (tops['items'][0]['uri'])
    print(top_track_uri)
    return top_track_uri

def get_recommendations(popularity, seed_g, seed_track_uri):
    albums = []
    results = sp.recommendations(seed_genres= seed_g, seed_tracks = [seed_track_uri], min_danceability=0, max_loudness=0, target_popularity = popularity, limit=10)
    #print(results)
    for track in results['tracks']:
        print (track['name'], '-', track['artists'][0]['name'])
        #tr = track['album']['id']

    return results['tracks'][0]['album']['id']

def get_available_devices():
    albums = []
    results = sp.devices()
    #print(results)
    return results

def find_RPi_Device_ID():
    devices = get_available_devices()["devices"]
    
    for dev in devices:
        #print(dev["id"])
        if dev["name"] == "raspotify (raspberrypi)":
            #print("found it")
            RPi_id = dev["id"]
    return RPi_id
    
def transfer_on_raspberryPi(id):
    sp.transfer_playback(id)

def play_track_id(device_id, track_id):
    track_url = "spotify:album:" + track_id
    sp.start_playback(device_id, track_url)

In [367]:
# gets top track
# from that , gets top artist
# from that , gets top genre
top_track_genre = get_current_user_top_track_genre()
top_artists_uri = get_current_user_top_artists_uri()
top_track_uri = get_current_user_top_track_uri()

# top genre, and top track are passed for seeding recommendations
track_id = get_recommendations(popularity, top_track_genre, top_track_uri)
#print(track_id)

# gets respberry pi device id and transfers playback to it
RapberryPi_Device_Id = find_RPi_Device_ID()
transfer_on_raspberryPi(RapberryPi_Device_Id)

#plays the track
play_track_id(RapberryPi_Device_Id, track_id)

spotify:track:2TEws3ZBMfsCZtuYL1k5KW
Behet Ghol Midam - Mohsen Yeganeh
To Ke Nisti Pisham - Masih
Hesse Khoobie - Shadmehr Aghili
Mano Negah Kon - Babak Jahanbakhsh
Asheghaneh - Farzad Farzin
Shabe Tire - Amir Tataloo
Dooset Daram - Mohsen Yeganeh
Goli - Masih
Taghdir - Shadmehr Aghili
Ey Vaaay - Babak Jahanbakhsh
